In [12]:
!nvidia-smi

Wed Dec 10 16:35:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              2W /   50W |       1MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

purpose of accelerate



In [13]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

##purpose of accelerate

*   ease of multidevice Training - whetyher you are using an  tpu or gpu with accelerate you cans cale yopur nmodel to multiple devices with minimal code changes
*   Mixed Precison - models are trained using kixed precision so that training is sped up and memory usage is reduced

*  Zero REdundancy optimize (ZeRO)- efficiently splits large  model among multiple devices



In [14]:
!pip uninstall -y transformers accelerate torch torchvision torchaudio
!pip install torch torchvision torchaudio
!pip install transformers accelerate

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0
Uninstalling torchaudio-2.6.0:
  Successfully uninstalled torchaudio-2.6.0
  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp313-cp313-win_amd64.whl.metadata (5.9 kB)
  Using cached torchaudio-2.9.1-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.9.1-cp313-cp313-win_amd64.whl (110.9 MB)
Using cached torchvision-0.24.1-cp313-cp313-

In [15]:
from transformers import  pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

from transformers import   AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
!pip install torch==2.6.0 torchvision torchaudio


^C


###basic working and loading of huggingface model


In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("transformersbook/pegasus-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("transformersbook/pegasus-samsum", from_tf=False)

ARTICLE_TO_SUMMARIZE = (
    "Human reproduction is the process of sexual reproduction involving the fusion of male sperm and female egg to create a new individual. It includes gamete formation (spermatogenesis and oogenesis), fertilization, implantation of the fertilized egg (zygote) in the uterus, embryonic development within the womb, and childbirth. This entire process is sexual in nature, requiring two parents with different sexes to contribute genetic material"
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'Human reproduction involves the fusion of male sperm and female egg to create a new individual. It includes gamete formation (spermatogenesis), fertilization, implantation of the fertilized egg in the uterus, embryonic development within the womb, and childbirth.'

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

device="cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

##fine Tuning


In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model="transformersbook/pegasus-samsum"
tokenizer = AutoTokenizer.from_pretrained(model)

# Ensure the tokenizer has a pad token, add if missing (though Pegasus typically has one)
#if tokenizer.pad_token is None:
  #  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model2 = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

# Explicitly set decoder_start_token_id and bos_token_id in the model's generation config.
# This is crucial for generation and attention mask creation with models like Pegasus,
# and helps resolve issues where these might be None or incorrectly aligned.
#model2.config.decoder_start_token_id = tokenizer.pad_token_id
#model2.config.bos_token_id = tokenizer.pad_token_id

In [21]:
!wget https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
!unzip summarizer-data.zip


'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
from datasets import load_from_disk
ds=load_from_disk("samsum_dataset")
ds

FileNotFoundError: Directory samsum_dataset not found

In [ ]:
def preprocess(example):
    model_inputs = tokenizer(
        example["dialogue"],
        max_length=256,   # <-- source truncation
        truncation=True,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["summary"],
            max_length=64,  # <-- target truncation
            truncation=True,
        )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

# Apply to all splits
ds1 = ds.map(preprocess, batched=True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
split_lengths = [len(ds[split])for split in ds]

print(f"Split lengths: {split_lengths}")
print(f"Features: {ds['train'].column_names}")
print("\nDialogue:")

print(ds["test"][4]["dialogue"])

print("\nSummary:")

print(ds["test"][4]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Ollie: Hi , are you in Warsaw
Jane: yes, just back! Btw are you free for diner the 19th?
Ollie: nope!
Jane: and the  18th?
Ollie: nope, we have this party and you must be there, remember?
Jane: oh right! i lost my calendar..  thanks for reminding me
Ollie: we have lunch this week?
Jane: with pleasure!
Ollie: friday?
Jane: ok
Jane: what do you mean " we don't have any more whisky!" lol..
Ollie: what!!!
Jane: you just call me and the all thing i heard was that sentence about whisky... what's wrong with you?
Ollie: oh oh... very strange! i have to be carefull may be there is some spy in my mobile! lol
Jane: dont' worry, we'll check on friday.
Ollie: don't forget to bring some sun with you
Jane: I can't wait to be in Morocco..
Ollie: enjoy and see you friday
Jane: sorry Ollie, i'm very busy, i won't have time for lunch  tomorrow, but may be at 6pm after my courses?this trip to Morocco was so nice, but time 

In [ ]:
def cvrt_examples_to_features(ex_batch):
  # Change max_length to 511 to ensure it's within the model's max_position_embeddings (512) for 1-indexed positions
  input_encodings=tokenizer(ex_batch['dialogue'], max_length=511, truncation=True)

  with tokenizer.as_target_tokenizer():
    # Ensure target sequence length also respects model limits, 180 is fine as it's < 511
    target_encodings=tokenizer(ex_batch['summary'], max_length=180, truncation=True)

  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask' :input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']

        }

In [ ]:
ds1=ds.map(cvrt_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
ds1['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

###
DataCollatorForSeq2Seq is a special data collator designed for sequence-to-sequence models (e.g., Pegasus, T5, BART) that helps in preparing batches of data for training.

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer , model2)

In [ ]:
!pip install --upgrade transformers


In [ ]:
def fix_labels(example):
    example["labels"] = [
        -100 if tok == tokenizer.pad_token_id else tok
        for tok in example["labels"]
    ]
    return example

ds1 = ds1.map(fix_labels)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

from transformers import TrainingArguments

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    warmup_steps=200,
    learning_rate=3e-5,
    logging_steps=50,

    fp16=True,

    save_strategy="no",
    report_to="none",
)


In [ ]:
labels = torch.tensor(ds1["train"][0]["labels"])
print(labels.dtype)





torch.int64


In [ ]:
trainer = Trainer(model=model2, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=ds1["test"],
                  eval_dataset=ds1["validation"])

/tmp/ipython-input-746978280.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model2, args=trainer_args,


In [ ]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
50,3.355200
100,2.986600
150,2.439400
200,2.139300
250,2.024300
300,2.099300
350,1.727100
400,1.903600


TrainOutput(global_step=410, training_loss=2.32410208074058, metrics={'train_runtime': 209.4056, 'train_samples_per_second': 3.911, 'train_steps_per_second': 1.958, 'total_flos': 313354514841600.0, 'train_loss': 2.32410208074058, 'epoch': 1.0})

In [ ]:
# Evaluation
### lst[1,2,3,4,5,6]-> [1,2,3][4,5,6]
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
                for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=20257aa912c85059ed52b3ac1236933a11f10760a4044ce9976901f2d7411c26
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import evaluate

rouge_metric = evaluate.load('rouge')
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

# Directly use the scores without accessing fmeasure or mid
rouge_dict = {rn: score[rn] for rn in rouge_names}

# Convert the dictionary to a DataFrame for easy visualization
import pandas as pd
pd.DataFrame(rouge_dict, index=[f'pegasus'])
